In [1]:
import pandas as pd
import googlemaps
import csv

## Bike Data
df_bikes = pd.read_csv('../../data/philadelphia_2016.csv')

#Get Station id's Mapped to Name
unique_station_ids = df_bikes['start_station_id'].fillna(0).unique()

In [2]:
#To check if there are no multiple names for start and End (Has to be true, so every Station ID Corresponds to a Unique Name
all([all(df_bikes.groupby(df_bikes['start_station_id']).nunique()['start_station_name']),
all(df_bikes.groupby(df_bikes['end_station_id']).nunique()['end_station_name'])])

True

Extract all Start and End functions from the Data

In [3]:
start_station_tuples = df_bikes.groupby(['start_station_id','start_station_name']).max().index
end_station_tuples = df_bikes.groupby(['end_station_id','end_station_name']).max().index

both_station_tuples = set(list(start_station_tuples) + list(end_station_tuples))

C:\Users\novem\AppData\Local\Temp\ipykernel_10176\876552045.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  end_station_tuples = df_bikes.groupby(['end_station_id','end_station_name']).max().index


Add Philadelphia to the list to make it more specific for Google

In [4]:
#Add Philadelphia to make it easier for Google
stations = []

for station in both_station_tuples:
    station_as_list = list(station)
    station_as_list[1] += ", Philadelphia, PA"
    station_as_list[0] = int(station_as_list[0])
    stations.append(station_as_list)

Works with API Key, but is paid with limited free tries, so I have not pushed it to GIT
Call the GMaps API and save the datapoints to a List

In [5]:
fetched_stations = []

gmaps = googlemaps.Client(key='api_key')
"""
This for loop makes an API Request for every Name, and fetches the lat and lng, and then writes it to a CSV
"""
for station in stations:
    row = gmaps.geocode(station[1])
    lat = row[0]["geometry"]["location"]["lat"]
    lng = row[0]["geometry"]["location"]["lng"]
    fetched_stations.append({"lat": lat, "lon": lng, "name": station[1], "id": station[0]})

ValueError: Invalid API key provided.

Write everything to it's own CSV

In [ ]:
with open('../../data/stations.csv', 'w', newline="") as myfile:
    writer = csv.DictWriter(myfile, fetched_stations[0].keys())
    writer.writeheader()
    writer.writerows(fetched_stations)